In [1]:
import galois
import numpy as np

from py_ecc.optimized_bn128.optimized_curve import multiply, G1, G2, add, normalize, neg, FQ12
from py_ecc.optimized_bn128.optimized_pairing import pairing

In [2]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

### R1CS

In [3]:
# inputs
x = FP(2)
y = FP(3)

# witnesses
v1 = x * x # (1)
v2 = y * y # (2)
v3 = 5 * x * v1 # (3)
v4 = 4 * v1 * v2 # (4)
out = 13 * x * v2 - 10 * y + v1 + v3 - v4 # (5)

# check if computed output is correct
assert out == 5 * x**3 - 4 * x**2 * y**2 + 13 * x * y**2 + x**2 - 10 * y

# witness vector
s = FP([1, out, x, y, v1, v2, v3, v4])

L = FP([
    # 1, out, x, y, v1, v2, v3, v4
    [0, 0,  1, 0, 0, 0, 0, 0], # (1) x
    [0, 0,  0, 1, 0, 0, 0, 0], # (2) y
    [0, 0,  5, 0, 0, 0, 0, 0], # (3) 5x
    [0, 0,  0, 0, 4, 0, 0, 0], # (4) 4v1
    [0, 0, 13, 0, 0, 0, 0, 0], # (5) 13x
])

R = FP([
    [0, 0, 1, 0, 0, 0, 0, 0], # x
    [0, 0, 0, 1, 0, 0, 0, 0], # y
    [0, 0, 0, 0, 1, 0, 0, 0], # v1
    [0, 0, 0, 0, 0, 1, 0, 0], # v2
    [0, 0, 0, 0, 0, 1, 0, 0], # v2
])

O = FP([
    [0, 0, 0,  0,   1, 0,   0, 0], # v1
    [0, 0, 0,  0,   0, 1,   0, 0], # v2
    [0, 0, 0,  0,   0, 0,   1, 0], # v3
    [0, 0, 0,  0,   0, 0,   0, 1], # v4
    [0, 1, 0, 10, p-1, 0, p-1, 1], # out + 10y - v1 - v3 + v4
])

Ls = np.dot(L, s)
Rs = np.dot(R, s)
Os = np.dot(O, s)

assert(np.all(Ls * Rs == Os))

### QAP

In [4]:
# num witnesses
m = O.shape[1]
# num constraints
d = O.shape[0]

print(f"m: {m}, d: {d}")

poly_m = []

# iterate over the matrixes
for M in [L, R, O]:
    poly_list = []
    # iterate over the columns (witnesses)
    for i in range(0, m):
        # there must be `d` number of pairs (x, y) to interpolate
        # one for each constraint
        points_x = FP(np.zeros(d, dtype=int))
        points_y = FP(np.zeros(d, dtype=int))
        # iterate over the rows (constraints)
        for j in range(0, d):
            # x coordinate is the index of the row + 1
            points_x[j] = FP(j + 1)
            # y coordinate is the value of the matrix at (i, j)
            points_y[j] = M[j][i]

        # making polynomial interpolation (must be d-1 degree polynomial, since we have d points)
        poly = galois.lagrange_poly(points_x, points_y)
        # we need only the coefficients of the polynomial (in ascending order, c0*x^0, c1*x^1, ...)
        coeffs = poly.coefficients(order="asc")
        
        # if the polynomial is smaller than d, append zeros for higher coefficients
        if len(coeffs) < d:
            coeffs = np.append(coeffs, np.zeros(d - len(coeffs), dtype=int))
        
        poly_list.append(coeffs)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

U = galois.Poly(np.matmul(s, Lp), order="asc")
V = galois.Poly(np.matmul(s, Rp), order="asc")
W = galois.Poly(np.matmul(s, Op), order="asc")

print("U = ", U)
print("V = ", V)
print("W = ", W)

T = galois.Poly([1, p-1], field=FP)
for i in range(2, d + 1):
    T *= galois.Poly([1, p-i], field=FP)

H = (U * V - W) // T
rem = (U * V - W) % T

assert rem == 0, "must not be the remainder"

print("T = ", T)
print("H = ", H)

tau = FP(20)

assert U(tau) * V(tau) - W(tau) == H(tau) * T(tau)

m: 8, d: 5
U =  10944121435919637611123202872628637544274182200208017171849102093287904247809x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415930x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904247836x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079635x + 26
V =  11856131555579607412050136445347690672963697383558685269503193934395229601792x^4 + 20064222632519335620392538599819168831169334033714698148390020504361157787655x^3 + 20976232752179305421319472172538221959858849217065366246044112345468483141610x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955812x + 21888242871839275222246405745257275088548364400416034343698204186575808495601
W =  12768141675239577212977070018066743801653212566909353367157285775502554955771x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831936x^3 + 912010119659969800926933572719053128689515183350668097654091841107325

$\left\langle s \right\rangle_{[m]} \text{ - witness vector}\\$

$L_P = \begin{bmatrix}
& \color{Cyan}x^0 & \color{Cyan}x^1 & ... & \color{Cyan} x^{d-1} \\
\color{Cyan}s_0 & l_{0,0} & l_{0,1} & ... & l_{0, d-1} \\
\color{Cyan}s_1 & l_{1,0} & l_{1,1} & ... & l_{1, d-1} \\
\vdots & & & \ddots & \\
\color{Cyan}s_{m-1} & l_{m,0} & l_{m,1} & ... & l_{m, d-1} \\
\end{bmatrix}
R_P = \begin{bmatrix}
& \color{Cyan}x^0 & \color{Cyan}x^1 & ... & \color{Cyan} x^{d-1} \\
\color{Cyan}s_0 & r_{0,0} & r_{0,1} & ... & r_{0, d-1} \\
\color{Cyan}s_1 & r_{1,0} & r_{1,1} & ... & r_{1, d-1} \\
\vdots & & & \ddots & \\
\color{Cyan}s_{m-1} & r_{m,0} & r_{m,1} & ... & r_{m, d-1} \\
\end{bmatrix}
O_P = \begin{bmatrix}
& \color{Cyan}x^0 & \color{Cyan}x^1 & ... & \color{Cyan} x^{d-1} \\
\color{Cyan}s_0 & o_{0,0} & o_{0,1} & ... & o_{0, d-1} \\
\color{Cyan}s_1 & o_{1,0} & o_{1,1} & ... & o_{1, d-1} \\
\vdots & & & \ddots & \\
\color{Cyan}s_{m-1} & o_{m,0} & o_{m,1} & ... & o_{m, d-1} \\
\end{bmatrix}\\$

$U: \left\{ 
\begin{array}{l}
\left\langle L_{p_i}(x) \right\rangle_{i\in\{0..m-1\}} = \sum_{j=0}^{d-1}L_{P_{i, j}}*x^j \\
U(x) = \sum_{i=0}^{m-1}L_{p_i}(x)*s_i\ = \sum_{i=0}^{m-1}U_i(x) \color{Cyan} = \sum_{i=0}^{d-1}u_i*x^i=\left\langle u_0, u_1, ..., u_{d-1} \right\rangle \\
\end{array}
\right.\\$

$V: \left\{ 
\begin{array}{l}
\left\langle R_{p_i}(x) \right\rangle_{i\in\{0..m-1\}} = \sum_{j=0}^{d-1}R_{P_{i, j}}*x^j \\
V(x) = \sum_{i=0}^{m-1}R_{p_i}(x)*s_i  = \sum_{i=0}^{m-1}V_i(x) \color{Cyan} = \sum_{i=0}^{d-1}v_i*x^i=\left\langle v_0, v_1, ..., v_{d-1} \right\rangle \\
\end{array}
\right.\\$

$W: \left\{ 
\begin{array}{l}
\left\langle O_{p_i}(x) \right\rangle_{i\in\{0..m-1\}} = \sum_{j=0}^{d-1}O_{P_{i, j}}*x^j \\
W(x) = \sum_{i=0}^{m-1}O_{p_i}(x)*s_i  = \sum_{i=0}^{m-1}W_i(x)\color{Cyan} = \sum_{i=0}^{d-1}w_i*x^i=\left\langle w_0, w_1, ..., w_{d-1} \right\rangle \\
\end{array}
\right.\\$

$T(x) = \prod_{i=1}^{d}(x-i)\\$

$H(x) = \frac{U(x) \cdot V(x) - W(x)}{T(x)}\color{Cyan} = \sum_{i=0}^{d-1}h_i*x^i=\left\langle h_0, h_1, ..., h_{d-1} \right\rangle\\$

---

$U(x) * V(x) = W(x) + H(x)*T(x)\\$

### Protocol

Random values: $\tau, \alpha, \beta, \delta, \gamma, r, q\\$

$m = m_{pub} + m_{priv}\\$

$\left\langle s \right\rangle_{m} = \left\langle s_0, s_1, \cdots, s_{m_{pub} - 1}  \right\rangle_{m_{pub}} + \left\langle s_{m_{pub}}, s_{m_{pub} + 1}, \cdots, s_{m - 1} \right\rangle_{m_{priv}}\\$

$\left\langle \beta_L \right\rangle_{m\text{x}d} = \beta * L_P\\$

$\left\langle \alpha_R \right\rangle_{m\text{x}d} = \alpha * R_P\\$

$\left\langle K_P \right\rangle_{m\text{x}d} = \beta_L + \alpha_R + O_P\\$

$\left\langle K_{p_i}(\tau) \right\rangle_{i\in\{0..m-1\}} = \sum_{j=0}^{d-1}K_{P_{i, j}}*\tau^j\\$

$\left\{ 
\begin{array}{l}
U_{Pub}(x) = \sum_{0}^{m_{pub} - 1} U_i(x) \\
U_{Priv}(x) = \sum_{m_{pub}}^{m - 1} U_i(x) \\
U(x) = U_{Pub}(x) + U_{Priv}(x)\\
\end{array}
\right.
\left\{ 
\begin{array}{l}
V_{Pub}(x) = \sum_{0}^{m_{pub} - 1} V_i(x) \\
V_{Priv}(x) = \sum_{m_{pub}}^{m - 1} V_i(x) \\
V(x) = V_{Pub}(x) + V_{Priv}(x)\\
\end{array}
\right.
\left\{ 
\begin{array}{l}
W_{Pub}(x) = \sum_{0}^{m_{pub} - 1} W_i(x) \\
W_{Priv}(x) = \sum_{m_{pub}}^{m - 1} W_i(x) \\
W(x) = W_{Pub}(x) + W_{Priv}(x)\\
\end{array}
\right.\\$

$\left\{ 
\begin{array}{l}
K_{Pub} = \sum_{i=0}^{m_{pub} - 1}s_i*K_{p_i}(\tau) \color{Cyan} = \sum_{i=0}^{m_{pub} - 1}\beta*U_i(\tau)+\alpha*V_i(\tau)+W_i(\tau) =  \beta*U_{Pub}(\tau) + \alpha*V_{Pub}(\tau) + W_{Pub}(\tau)\\
K_{Priv} = \sum_{i=m_{pub}}^{m - 1}s_i*K_{p_i}(\tau) \color{Cyan} = \sum_{i=m_{pub}}^{m - 1}\beta*U_i(\tau)+\alpha*V_i(\tau)+W_i(\tau) = \beta*U_{Priv}(\tau) + \alpha*V_{Priv}(\tau) + W_{Priv}(\tau)\\
K = K_{Pub} + K_{Priv} \color{Cyan} = \sum_{i=0}^{m - 1}\beta*U_i(\tau)+\alpha*V_i(\tau)+W_i(\tau) = \beta*U(\tau) + \alpha*V(\tau) + W(\tau)
\end{array}
\right.\\$

$\left\{ 
\begin{array}{l}
A = U(\tau) + \alpha + r * \delta\\
B = V(\tau) + \beta + q * \delta\\
C = \frac{K_{Priv} + H(\tau)*T(\tau)}{\delta} + q * A + r * B - r * q * \delta = \frac{\beta*U_{Priv}(\tau) + \alpha*V_{Priv}(\tau) + W_{Priv}(\tau) + H(\tau)*T(\tau)}{\delta} + q * A + r * B - r * q * \delta\\
\frac{K_{Pub}}{\gamma} = \frac{\beta*U_{Pub}(\tau) + \alpha*V_{Pub}(\tau) + W_{Pub}(\tau)}{\gamma}\\
A * B = \alpha\beta + C * \delta + \frac{K_{pub}}{\gamma} * \gamma\\
\end{array}
\right.\\$

---

PoC:

$A * B = \alpha\beta + C * \delta + \frac{K_{pub}}{\gamma} * \gamma\\$

$A * B = \alpha\beta + \left[\frac{K_{Priv} + HT}{\delta} + qA + rB - rq\delta \right] * \delta + \frac{K_{pub}}{\gamma} * \gamma\\$

$A * B = \alpha\beta + K_{Priv} + HT + q\delta A + r\delta B - rq\delta^2 + K_{Pub}\\$

$A * B = \alpha\beta + K + HT + q\delta A + r\delta B - rq\delta^2\\$

$(U(\tau) + \alpha + r\delta) * (V(\tau) + \beta + q\delta) = \alpha\beta + \beta U(\tau) + \alpha V(\tau) + W(\tau) + H(\tau)*T(\tau) + q\delta  * (U(\tau) + \alpha + r\delta) + r\delta * (V(\tau) + \beta + q\delta) - rq\delta^2\\$

$U(\tau)*V(\tau) + \beta U(\tau) + q \delta U(\tau) + \alpha V(\tau) + \alpha\beta + q \alpha \delta + r \delta V(\tau) + r \beta \delta + rq\delta^2 = \alpha\beta + \beta U(\tau) + \alpha V(\tau) + W(\tau) + H(\tau)*T(\tau) + q\delta  * (U(\tau) + \alpha + r\delta) + r\delta * (V(\tau) + \beta + q\delta) - rq\delta^2\\$

$U(\tau)*V(\tau) + q \delta U(\tau) + \alpha\beta + q \alpha \delta + r \delta V(\tau) + r \beta \delta + rq\delta^2 = \alpha\beta + W(\tau) + H(\tau)*T(\tau) + q\delta U(\tau) + q\delta \alpha + q r\delta^2 + r\delta V(\tau) + r\delta \beta + rq\delta^2 - rq\delta^2\\$

$U(\tau)*V(\tau) = W(\tau) + H(\tau)*T(\tau)\\$

In [5]:
def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2
    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

tau = FP(20)
T_tau = T(tau)

r = FP(12)
q = FP(13)
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)

u = U(tau)
v = V(tau)
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

a = u + alpha + r * delta
b = v + beta + q * delta

c = ((beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1) + q * a + r * b - r * q * delta
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

assert a * b == alpha * beta + k * gamma + c * delta # should be equal

### Trusted setup

$\color{Green}[\alpha]_{G_1} \color{defaultcolor} = \alpha * G_1\\$

$\color{Green}[\beta]_{G_1} \color{defaultcolor} = \beta * G_1\\$

$\color{Green}[\beta]_{G_2} \color{defaultcolor} = \beta * G_2\\$

$\color{Green}[\gamma]_{G_2} \color{defaultcolor} = \gamma * G_2\\$

$\color{Green}[\delta]_{G_1} \color{defaultcolor} = \delta * G_1\\$

$\color{Green}[\delta]_{G_2} \color{defaultcolor} = \delta * G_2\\$

$\left\langle \color{Green}[\tau^i]_{G_1}\color{defaultcolor} \right\rangle_{i\in\{0..d-1\}} = G_1*\tau^i\\$

$\left\langle \color{Green}[\tau^i]_{G_2}\color{defaultcolor} \right\rangle_{i\in\{0..d-1\}} = G_2*\tau^i\\$

$\left\langle \color{Green}\left[ \frac{T(\tau)\tau^i}{\delta} \right]_{G_1}\color{defaultcolor} \right\rangle_{i\in\{0..d-1\}} = G_1 * \frac{T(\tau)*\tau^i}{\delta}\\$

$\left\langle \color{Green}\left[\frac{K_{p_i}(\tau)}{\gamma}\right]_{G_1}\color{defaultcolor} \right\rangle_{i\in\{0..m_{pub}-1\}} = G_1 * \frac{K_{p_i}(\tau)}{\gamma} \\$

$\left\langle \color{Green}\left[\frac{K_{p_i}(\tau)}{\delta}\right]_{G_1}\color{defaultcolor} \right\rangle_{i\in\{m_{priv}..m-1\}} = G_1 * \frac{K_{p_i}(\tau)}{\delta} \\$

In [6]:
def evaluate_poly_list(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append(galois.Poly(mtx[i], order="asc"))
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

print("Setup phase")
print("-"*10, "\n")

print("Toxic waste:")
print("-"*10)

alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

beta_L = beta * Lp
alpha_R = alpha * Rp
K = beta_L + alpha_R + Op # preimage of [βA + αB + C]

Kp = to_poly(K)
print_poly("Kp", Kp)

print("\nKp evaluations at tau:")
Kp_tau = evaluate_poly_list(Kp, tau)
print([int(k) for k in Kp_tau])

# G1[τ^0 * T(τ) / δ], G1[τ^1 * T(τ) / δ], ..., G1[τ^d-1 * T(τ) / δ
target_G1 = [multiply(G1, int((tau**i * T_tau) / delta)) for i in range(0, T.degree - 1)]

assert len(target_G1) == len(H.coefficients()), f"target_G1 length mismatch! {len(target_G1)} != {len(H.coefficients())}"

K_gamma_G1 = [multiply(G1, int(k / gamma)) for k in Kp_tau[:2]]
K_delta_G1 = [multiply(G1, int(k / delta)) for k in Kp_tau[2:]]

tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]
alpha_G1 = multiply(G1, int(alpha))
beta_G2 = multiply(G2, int(beta))
gamma_G2 = multiply(G2, int(gamma))
delta_G2 = multiply(G2, int(delta))
delta_G1 = multiply(G1, int(delta))
beta_G1 = multiply(G1, int(beta))

print("")
print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G2 = {normalize(beta_G2)}")
print(f"[γ]G2 = {normalize(gamma_G2)}")
print(f"[δ]G2 = {normalize(delta_G2)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")
print(f"[τT(τ)/δ]G1 = {[normalize(point) for point in target_G1]}")
print(f"[K/γ]G1 = {[normalize(point) for point in K_gamma_G1]}")
print(f"[K/δ]G1 = {[normalize(point) for point in K_delta_G1]}")
print(f"[β]G1 = {normalize(beta_G1)}")
print(f"[δ]G1 = {normalize(delta_G1)}")

Setup phase
---------- 

Toxic waste:
----------
α = 2
β = 3
γ = 4
δ = 5
τ = 20

Kp polynomials:
Kp_0 = 0
Kp_1 = 20976232752179305421319472172538221959858849217065366246044112345468483141633x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539840x^3 + 11856131555579607412050136445347690672963697383558685269503193934395229601794x^2 + 1824020239319939601853867145438106257379030366701336195308183682214650707966x + 1
Kp_2 = 9120101196599698009269335727190531286895151833506680976540918411073253539846x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415872x^3 + 12768141675239577212977070018066743801653212566909353367157285775502554956032x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079275x + 214
Kp_3 = 9120101196599698009269335727190531286895151833506680976540918411073253539840x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831879x^3 + 12768141675239577212977070018066743801653212566

### Proof generation

Prover picks random: $r$, $q$

$\left[\frac{HT}{\delta}\right]_{G_1} = \left[\frac{H(\tau)*T(\tau)}{\delta}\right]_{G_1} = \sum_{i=0}^{d-1}h_i * \color{Green}\left[ \frac{T(\tau)\tau^i}{\delta} \right]_{G_1} \color{Cyan} = \frac{T(\tau)}{\delta} * \sum_{i=0}^{d-1}h_i*\tau^i = \frac{H(\tau)*T(\tau)}{\delta}\\$

$\left[\frac{K_{Priv}}{\delta}\right]_{G_1} = \sum_{i=m_{pub}}^{m-1} s_i * \color{Green}\left[\frac{K_i(\tau)}{\delta}\right]_{G_1} \color{Cyan} = \frac{1}{\delta} * \sum_{i=m_{pub}}^{m-1} s_i *[\beta*U_i(\tau)+\alpha * V_i(\tau)+W_i(\tau)] = \frac{\beta*U_{Priv}(\tau) + \alpha*V_{Priv}(\tau)+W_{Priv}(\tau)}{\delta}\\$

$[r\delta]_{G_1} = r * \color{Green}[\delta]_{G_1}\\$

$[q\delta]_{G_1} = q * \color{Green}[\delta]_{G_1}\\$

$[q\delta]_{G_2} = q * \color{Green}[\delta]_{G_2}\\$

$[qA]_{G_1} = q * [A]_{G_1}\\$

$[rB]_{G_1} = r * [B]_{G_1}\\$

$[-rq\delta]_{G_1} = -r*q*\color{Green}[\delta]_{G_1}\\$

$[B]_{G_1} = [V(\tau)]_{G_1} + \color{Green}\left[ \beta \right]_{G_1} \color{defaultcolor} + [q\delta]_{G_1} = \sum_{i=0}^{d-1}v_i*\color{Green}[\tau^i]_{G_1} \color{defaultcolor} + \color{Green}\left[ \beta\right]_{G_1}\color{defaultcolor} + [q\delta]_{G_1} \color{Cyan} = V(\tau) + \beta + q * \delta \\$

---

$[A]_{G_1} = [U(\tau)]_{G_1} + \color{Green}\left[ \alpha \right]_{G_1} \color{defaultcolor} + [r\delta]_{G_1} = \sum_{i=0}^{d-1}u_i*\color{Green}[\tau^i]_{G_1} \color{defaultcolor} + \color{Green}\left[ \alpha \right]_{G_1}\color{defaultcolor} + [r\delta]_{G_1} \color{Cyan} = U(\tau) + \alpha + r*\delta \\$

$[B]_{G_2} = [V(\tau)]_{G_2} + \color{Green}\left[ \beta \right]_{G_2} \color{defaultcolor} + [q\delta]_{G_2} = \sum_{i=0}^{d-1}v_i*\color{Green}[\tau^i]_{G_2} \color{defaultcolor} + \color{Green}\left[ \beta\right]_{G_2}\color{defaultcolor} + [q\delta]_{G_2} \color{Cyan} = V(\tau) + \beta + q * \delta\\$

$[C]_{G_1} = \left[\frac{HT}{\delta}\right]_{G_1} + \left[\frac{K_{Priv}}{\delta}\right]_{G_1} + [qA]_{G_1} + [rB]_{G_1} + [-rq\delta]_{G_1} \color{Cyan} = \frac{H(\tau)*T(\tau) + \beta*U_{Priv}(\tau) + \alpha*V_{Priv}(\tau)+W_{Priv}(\tau)}{\delta} + q*A + r*B - r*q*\delta\\$

In [7]:
def evaluate_poly(poly, trusted_points):
    coeffs = poly.coefficients(order="asc")

    assert len(coeffs) == len(trusted_points), "Polynomial degree mismatch!"

    # encoding the polynomial's coefficients as points
    # P[coeffs[0]] + P[coeffs[1]] + ... + P[coeffs[-1]]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeffs)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    return evaluation

r = FP(12) # should be random
q = FP(13) # should be random

pub_input, priv_input = s[:2], s[2:]

print("Proof generation:")
print("-"*10)

k_G1 = [multiply(G1, int(k)) for k in Kp_tau]

k_pub_G1, k_priv_G1 = k_G1[:2], k_G1[2:]

r_delta_G1 = multiply(delta_G1, int(r))
q_delta_G1 = multiply(delta_G1, int(q))
q_delta_G2 = multiply(delta_G2, int(q))

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
A_G1 = add(A_G1, r_delta_G1)

B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
B_G2 = add(B_G2, q_delta_G2)

B_G1 = evaluate_poly(V, tau_G1)
B_G1 = add(B_G1, beta_G1)
B_G1 = add(B_G1, q_delta_G1)

As_G1 = multiply(A_G1, int(q))
Br_G1 = multiply(B_G1, int(r))
rq_delta_G1 = multiply(delta_G1, int(-r*q))

HT_G1 = evaluate_poly(H, target_G1)

K_priv_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_delta_G1, priv_input)]
K_priv_G1 = K_priv_G1_terms[0]
for i in range(1, len(K_priv_G1_terms)):
    K_priv_G1 = add(K_priv_G1, K_priv_G1_terms[i])

C_G1 = add(K_priv_G1, HT_G1)
C_G1 = add(C_G1, As_G1)
C_G1 = add(C_G1, Br_G1)
C_G1 = add(C_G1, rq_delta_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")

Proof generation:
----------
[A]G1 = (4146595707987628546178337541096465056995563369739737135364129040262944417496, 2521048881735618568027902637087222589485528054669409524449947354852573256885)
[B]G2 = ((5256258010290086352378320153637445171313958016499931388290625339205463103023, 12147871142219645045116563920363505725991961064686617590172416751606014848074), (13421139632461316868950368404623277182505122572304353233725263705634257888588, 13437601836937040072397794963547645583984650693374715121342721434753591088949))
[C]G1 = (18475679588137010955165041916086021561999625738567681319063195567091753877496, 7907911394844033338347634811446008136200405591027449215980067101001920127119)


### Verification

Given:

$\left\{ 
\begin{array}{l}
\left\langle s \right\rangle_{[m_{pub}]} & \text{public inputs (witnesses)}\\
[A]_{G_1}, [B]_{G_2}, [C]_{G_1} & \text{proof}\\
\end{array}
\right.\\$

Verifier calculates:

$\left[\frac{K_{Pub}}{\gamma}\right]_{G_1} = \sum_{i=0}^{m_{pub} - 1} s_i * \color{Green}\left[\frac{K_{p_i}(\tau)}{\gamma}\right]_{G_1}\\$

Verifier checks:

$e([A]_{G_1}, [B]_{G_2}) = e(\color{Green}[\alpha]_{G_1}\color{defaultcolor}, \color{Green}[\beta]_{G_2}\color{defaultcolor}) * e([C]_{G_1}, \color{Green}[\delta]_{G_2}\color{defaultcolor}) * e(\left[\frac{K_{Pub}}{\gamma}\right]_{G_1}, \color{Green}[\gamma]_{G_2}\color{defaultcolor})\\$

$\color{Cyan}[G_{T}]^{AB} = [G_{T}]^{\alpha\beta + C * \delta + \frac{K_{Pub}}{\gamma} * \gamma}\\$

$\color{Cyan}AB = \alpha\beta + C * {\delta} + \frac{K_{Pub}}{\gamma} * \gamma\\$

In [8]:
print("Proof verification:")
print("-"*10)

K_pub_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_gamma_G1, pub_input)]
K_pub_G1 = K_pub_G1_terms[0]
for i in range(1, len(K_pub_G1_terms)):
    K_pub_G1 = add(K_pub_G1, K_pub_G1_terms[i])

assert pairing(B_G2, A_G1) == pairing(beta_G2, alpha_G1) * pairing(delta_G2, C_G1) * pairing(gamma_G2, K_pub_G1)

print("Pairing check passed!")

Proof verification:
----------
Pairing check passed!
